In [67]:
def calculate_f1(p, r):
    return round(2*((p*r)/(p+r)),3)

In [68]:
def calculate(label, df):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    kosong = 0
    
    for index in range(0, len(df)):
        labels = df['label'][index].split(',')
        
        if type(df['predict'][index]) != float:
            preds = df['predict'][index].split('|')

            if label in labels and label in preds:
                tp += 1
            elif label not in labels and label not in preds:
                tn += 1
            elif label not in labels and label in preds:
                fp += 1
            elif label in labels and label not in preds:
                fn += 1
        elif label in labels:
            fn += 1
        else:
            tn += 1
    precision = round(tp / (tp + fp), 2)
    recall = round(tp / (tp + fn), 2)
    f1 = calculate_f1(precision, recall)
    accuracy = round((tp+tn)/(tp+tn+fp+fn), 2)
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    f = 2 * ((p * r) / (p + r))
    
    
    print(tp, '\t', tn, '\t', fp, '\t', fn, '\t', '\t', precision, '\t', recall, '\t', f1, '\t', '\t', accuracy)
    return tp, tn, fp, fn, p, r, f, accuracy

In [71]:
import pandas as pd
import numpy as np

def run(tipe):
    df = pd.read_csv('Results/Categorization/'+ tipe +'.csv')
    sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1', 'accuracy'])
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', 'prec', '\t', 'rec', '\t', 'f1', '\t', '\t', 'accuracy')
    precision = []
    recall = []
    f1 = []
    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'PRICES']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, df)
        precision.append(e)
        recall.append(f)
        f1.append(g)
        sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
    sf.to_excel("Results/Calculation/"+ tipe +".xlsx")
#     print(precision)
    print('', '\t', '', '\t', '', '\t', '', '\t', '\t', round(np.mean(precision),2), '\t', round(np.mean(recall),2), '\t', calculate_f1(round(np.mean(precision),2), round(np.mean(recall),2)), '\t', '\t')
    return np.mean(precision), np.mean(recall), calculate_f1(np.mean(precision), np.mean(recall))


In [60]:
run("lda2vec-elmo-money")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
105 	 237 	 74 	 46 	 	 0.59 	 0.7 	 0.64 	 	 0.74
183 	 169 	 76 	 34 	 	 0.71 	 0.84 	 0.77 	 	 0.76
126 	 303 	 5 	 28 	 	 0.96 	 0.82 	 0.88 	 	 0.93
92 	 322 	 10 	 38 	 	 0.9 	 0.71 	 0.79 	 	 0.9
 	  	  	  	 	 0.79 	 0.77 	 0.78 	 	


(0.79, 0.77, 0.78)

In [64]:
run("lda2vec-partial-ner-elmo-money")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
118 	 241 	 70 	 33 	 	 0.63 	 0.78 	 0.697 	 	 0.78
199 	 157 	 88 	 18 	 	 0.69 	 0.92 	 0.789 	 	 0.77
129 	 299 	 9 	 25 	 	 0.93 	 0.84 	 0.883 	 	 0.93
93 	 318 	 14 	 37 	 	 0.87 	 0.72 	 0.788 	 	 0.89
 	  	  	  	 	 0.78 	 0.82 	 0.799 	 	


(0.78, 0.82, 0.799)

In [65]:
run("plsa-partial-ner-elmo-money")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
115 	 245 	 66 	 36 	 	 0.64 	 0.76 	 0.695 	 	 0.78
200 	 159 	 86 	 17 	 	 0.7 	 0.92 	 0.795 	 	 0.78
130 	 300 	 8 	 24 	 	 0.94 	 0.84 	 0.887 	 	 0.93
93 	 319 	 13 	 37 	 	 0.88 	 0.72 	 0.792 	 	 0.89
 	  	  	  	 	 0.79 	 0.81 	 0.8 	 	


(0.79, 0.81, 0.8)

In [33]:
run("lda2vec-elmo-money")

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
105 	 237 	 74 	 46 	 	 0.59 	 0.7 	 0.64 	 	 0.74
183 	 169 	 76 	 34 	 	 0.71 	 0.84 	 0.77 	 	 0.76
126 	 303 	 5 	 28 	 	 0.96 	 0.82 	 0.88 	 	 0.93
92 	 322 	 10 	 38 	 	 0.9 	 0.71 	 0.79 	 	 0.9
 	  	  	  	 	 0.79 	 0.77 	 0.78 	 	


(0.79, 0.77, 0.78)

In [72]:
ate = ['lda', 'lda2vec', 'plsa', 'dp']
sim = ['glove', 'w2v', 'fasttext', 'elmo', 'elmo_ori']

rf = pd.DataFrame(columns=['sim', 'ate', 'ac', 'precision', 'recall', 'f1'])
for a in ate:
    for s in sim:
        print(a, s)
        p, r, f = run(a + '-' + s + '-money')
        rf = rf.append({'sim': s, 'ate': a, 'ac': 'AC1', 'precision': p, 'recall': r, 'f1': f}, ignore_index=True)
        
#         p, r, f = run(a + '-partial-' + s)
#         rf = rf.append({'sim': s, 'ate': a, 'ac': 'AC2', 'precision': p, 'recall': r, 'f1': f}, ignore_index=True)
        
        p, r, f = run(a + '-partial-ner-' + s + '-money')
        rf = rf.append({'sim': s, 'ate': a, 'ac': 'AC3', 'precision': p, 'recall': r, 'f1': f}, ignore_index=True)

rf.to_excel("Results/all_method.xlsx")

lda glove
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
100 	 203 	 108 	 51 	 	 0.48 	 0.66 	 0.556 	 	 0.66
179 	 198 	 47 	 38 	 	 0.79 	 0.82 	 0.805 	 	 0.82
103 	 232 	 76 	 51 	 	 0.58 	 0.67 	 0.622 	 	 0.73
111 	 238 	 94 	 19 	 	 0.54 	 0.85 	 0.66 	 	 0.76
 	  	  	  	 	 0.6 	 0.75 	 0.667 	 	
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
115 	 201 	 110 	 36 	 	 0.51 	 0.76 	 0.61 	 	 0.68
190 	 189 	 56 	 27 	 	 0.77 	 0.88 	 0.821 	 	 0.82
113 	 234 	 74 	 41 	 	 0.6 	 0.73 	 0.659 	 	 0.75
114 	 232 	 100 	 16 	 	 0.53 	 0.88 	 0.662 	 	 0.75
 	  	  	  	 	 0.61 	 0.81 	 0.696 	 	
lda w2v
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
95 	 201 	 110 	 56 	 	 0.46 	 0.63 	 0.532 	 	 0.64
186 	 148 	 97 	 31 	 	 0.66 	 0.86 	 0.747 	 	 0.72
104 	 262 	 46 	 50 	 	 0.69 	 0.68 	 0.685 	 	 0.79
103 	 267 	 65 	 27 	 	 0.61 	 0.79 	 0.688 	 	 0.8
 	  	  	  	 	 0.61 	 0.74 	 0.669 	 	
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
110 	 201 	 110 	 41 	 	 0.5 	

 	  	  	  	 	 0.79 	 0.81 	 0.8 	 	
plsa elmo_ori
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
104 	 167 	 144 	 47 	 	 0.42 	 0.69 	 0.522 	 	 0.59
139 	 200 	 45 	 78 	 	 0.76 	 0.64 	 0.695 	 	 0.73
130 	 304 	 4 	 24 	 	 0.97 	 0.84 	 0.9 	 	 0.94
100 	 291 	 41 	 30 	 	 0.71 	 0.77 	 0.739 	 	 0.85
 	  	  	  	 	 0.71 	 0.74 	 0.725 	 	
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
125 	 162 	 149 	 26 	 	 0.46 	 0.83 	 0.592 	 	 0.62
164 	 181 	 64 	 53 	 	 0.72 	 0.76 	 0.739 	 	 0.75
133 	 295 	 13 	 21 	 	 0.91 	 0.86 	 0.884 	 	 0.93
99 	 300 	 32 	 31 	 	 0.76 	 0.76 	 0.76 	 	 0.86
 	  	  	  	 	 0.71 	 0.8 	 0.752 	 	
dp glove
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
106 	 261 	 50 	 45 	 	 0.68 	 0.7 	 0.69 	 	 0.79
171 	 192 	 53 	 46 	 	 0.76 	 0.79 	 0.775 	 	 0.79
109 	 258 	 50 	 45 	 	 0.69 	 0.71 	 0.7 	 	 0.79
84 	 274 	 58 	 46 	 	 0.59 	 0.65 	 0.619 	 	 0.77
 	  	  	  	 	 0.68 	 0.71 	 0.695 	 	
tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accur

In [ ]:
run("plsa-partial-ner-money") # PLSA

In [ ]:
run("lda-partial-ner-money") # LDA

In [ ]:
run("dp-partial-ner-money") # DP

In [5]:
run("lda2vec-partial-elmo_ori") # LDA2VEC ELMO

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
126 	 145 	 166 	 25 	 	 0.43 	 0.83 	 0.57 	 	 0.59
166 	 178 	 67 	 51 	 	 0.71 	 0.76 	 0.73 	 	 0.74
132 	 296 	 12 	 22 	 	 0.92 	 0.86 	 0.89 	 	 0.93
80 	 286 	 46 	 50 	 	 0.63 	 0.62 	 0.62 	 	 0.79
 	  	  	  	 	 0.67 	 0.77 	 0.72 	 	


In [ ]:
run("lda2vec-partial-ner-money") # LDA2VEC ELMO

In [ ]:
run("lda2vec-partial-ner-fasttext-money") # FASTTEXT

In [ ]:
run("lda2vec-partial-ner-w2v-money") # WORD2VEC

In [ ]:
run("lda2vec-partial-ner-glove-money") # GLOVE

In [66]:
p = 0.81
r = 0.79
round(2*((p*r)/(p+r)),3)

0.8

In [ ]:
round(0.555, 2)